In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt

# 시뮬레이터 설정
sim = Aer.get_backend("aer_simulator")
shots = 1024
n_qubits = 3
T = 10  # 반복 횟수

# 초기 회로 구성
def create_initial_circuit(thetas):
    qc = QuantumCircuit(n_qubits, n_qubits)
    for i, theta in enumerate(thetas):
        qc.ry(theta * np.pi, i)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

# 피드백 회로 구성
def apply_feedback(state):
    qc = QuantumCircuit(n_qubits)
    for i, bit in enumerate(state[::-1]):
        if bit == '1':
            qc.x(i)
    qc.barrier()
    return qc

# 회로 실행 함수
def run(qc):
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    return result.get_counts()

# 유사도 계산
def similarity(counts1, counts2):
    keys = set(counts1) | set(counts2)
    v1 = np.array([counts1.get(k, 0) for k in keys])
    v2 = np.array([counts2.get(k, 0) for k in keys])
    return 1 - cosine(v1, v2)

# 실험 루프
history = []
thetas = np.random.rand(n_qubits)
initial_circuit = create_initial_circuit(thetas)
counts = run(initial_circuit)
history.append(counts)

for t in range(1, T):
    dominant = max(counts, key=counts.get)
    fb_qc = apply_feedback(dominant)
    core_qc = create_initial_circuit(thetas)
    qc = fb_qc.compose(core_qc)
    counts = run(qc)
    history.append(counts)

# 외부 자극 삽입
stim_qc = apply_feedback(dominant)
stim_qc.rx(np.pi, 0)
stim_qc = stim_qc.compose(create_initial_circuit(thetas))
counts = run(stim_qc)
history.append(counts)

# 유사도 시각화
similarities = [
    similarity(history[i], history[i + 1]) for i in range(len(history) - 1)
]

plt.plot(similarities, marker='o')
plt.xlabel("Cycle")
plt.ylabel("Similarity to Previous")
plt.title("Structural Stability Over Feedback Loops")
plt.grid(True)
plt.show()
